## Get 5 road segments that have largest average travelling time

In [1]:
import pandas as pd

mr = pd.read_csv('data/mr.csv', sep=';', index_col=0)
mr.sort_index(inplace=True)
mr = mr.loc[mr['mgeom'] != 'LINESTRING()']
# mr['mgeom'] = mr['mgeom'].apply(shapely.wkt.loads)
# mr['pgeom'] = mr['pgeom'].apply(shapely.wkt.loads)
# print(len(mr['pgeom'].iloc[0].coords))
# print(len(mr['opath'].iloc[0].split(',')))
# print(len(mr['cpath'].iloc[0].split(',')))
mr.head()

,opath,pgeom,cpath,mgeom
id,,,,
1,"1095,1095,2732,2732,1093,1093,698,698,696,8687...","LINESTRING(-8.61862328377 41.1414555366,-8.618...","1095,2732,1093,698,8704,62550,8703,696,8687,17...","LINESTRING(-8.61862328377 41.1414555366,-8.618..."
2,"56205,56205,4790,70940,762,26201,26213,26202,5...","LINESTRING(-8.63985895545 41.159750087,-8.6403...","56205,69910,69917,69912,4790,56207,26241,70940...","LINESTRING(-8.63985895545 41.159750087,-8.6405..."
4,"5224,5224,5224,5224,46845,46845,46845,46846,46...","LINESTRING(-8.5747537 41.1518991,-8.5747537 41...","5224,46845,46861,46846,1004,1006,993,1015,3529...","LINESTRING(-8.5747537 41.1518991,-8.5743688 41..."
5,"60302,60302,67260,5878,5880,61719,765,70690,73...","LINESTRING(-8.6457599 41.1805278,-8.6457599 41...","60302,67260,57887,67265,5875,5878,5880,41236,6...","LINESTRING(-8.6457599 41.1805278,-8.6457599 41..."
6,"62561,58603,701,26471,55235,55235,55235,55235,...","LINESTRING(-8.61551679314 41.140715625,-8.6148...","62561,58603,58083,65094,57135,701,26471,55235,...","LINESTRING(-8.61551679314 41.140715625,-8.6154..."


In [2]:
road_segments = mr['opath'].tolist()
road_segments = ",".join(road_segments).split(",")
road_segments = [int(i) for i in road_segments]
road_segments.sort()

In [3]:
import collections
counter = collections.Counter(road_segments)
result = counter.most_common(5)
result = [i[0] for i in result]
print("5 FIDs of the road segments which are traversed most often:")
print(result)

5 FIDs of the road segments which are traversed most often:
[608, 54764, 148, 25119, 2481]


## Visualize these road segments

In [4]:
import osmnx as ox
import shapely.wkt

G = ox.load_graphml('data/porto-bbox.graphml')

edges = pd.read_csv('data/porto-bbox/edges.csv', index_col='fid')['WKT']
edges = edges[edges.index.isin(result)]
edges = edges.apply(shapely.wkt.loads)
edges


/Users/junr/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


fid
148      LINESTRING (-8.6248431 41.1732342, -8.62544499...
608      LINESTRING (-8.6678023 41.2363728, -8.66791560...
2481     LINESTRING (-8.6680157 41.2299263, -8.6679482 ...
25119    LINESTRING (-8.6695028 41.2377432, -8.6696046 ...
54764    LINESTRING (-8.652828400000001 41.2121418, -8....
Name: WKT, dtype: object

In [5]:
import numpy as np
coords = [np.array([list(i) for i in edges.iloc[j].coords]) for j in range(len(edges.index))]


In [6]:
import matplotlib.pyplot as plt

# Draw the basemap
fig, ax = ox.plot_graph(G, node_size=0, show=False, close=False, figsize=(48, 48))

# Get colors for each of the first 10 trajectories
color_list = ox.plot.get_colors(5, cmap='rainbow', return_hex=True)

# Plot the raw GPS points on the basemap
for idx, c in enumerate(color_list):
    plt.plot(*coords[idx].T, color=c, linewidth=4)

# Save the map as an image file
fig.savefig('output/task5-2.png', bbox_inches='tight', pad_inches=0)
# plt.show()
plt.close(fig)